<a href="https://colab.research.google.com/github/ulianakazak/Bigutochki/blob/main/%D0%91%D0%B5%D1%80%D1%82%2C%20%D0%BA%D0%BE%D1%82%D0%BE%D1%80%D1%8B%D0%B9%20%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%B5%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# coding: utf-8

# https://github.com/google-research/bert
# https://github.com/CyberZHG/keras-bert

# папка, куда распаковать преодобученную нейросеть BERT
folder = 'multi_cased_L-12_H-768_A-12'
download_url = 'https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip'  # ссылка на скачивание модели

print('Downloading model...')
zip_path = '{}.zip'.format(folder)
!test -d $folder || (wget $download_url && unzip $zip_path)

# скачиваем из BERT репозитория файл tokenization.py
!wget https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/tokenization.py

# install Keras BERT
!pip install keras-bert

import sys
import numpy as np
from keras_bert import load_trained_model_from_checkpoint
import tokenization

config_path = folder+'/bert_config.json'
checkpoint_path = folder+'/bert_model.ckpt'
vocab_path = folder+'/vocab.txt'

# создаем объект для перевода строки с пробелами в токены
tokenizer = tokenization.FullTokenizer(vocab_file=vocab_path, do_lower_case=False)

# загружаем модель
print('Loading model...')
model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True)
#model.summary()          # информация о слоях нейросети - количество параметров и т.д.
print('OK')


--2021-03-13 18:50:13--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 2607:f8b0:400e:c08::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip’

multi_cased_L-12_H- 100%[===================>] 632.19M   114MB/s    in 6.1s    

2021-03-13 18:50:20 (103 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip’ saved [662903077/662903077]

Archive:  multi_cased_L-12_H-768_A-12.zip
   creating: multi_cased_L-12_H-768_A-12/
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: m

# Новый раздел

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import fbeta_score,make_scorer

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords



In [ ]:
#train = pd.read_json('snli_1.0_train.jsonl', lines = True)
test = pd.read_json('snli_1.0_test.jsonl', lines = True)
#val = pd.read_json('snli_1.0_dev.jsonl', lines = True)
#full = {'train': train, 'val':val, 'test': test}
#full = {'val':val, 'test': test}

In [ ]:
test.head()

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,"[neutral, contradiction, contradiction, neutra...",2677109430.jpg#1,neutral,2677109430.jpg#1r1n,This church choir sings to the masses as they ...,( ( This ( church choir ) ) ( ( ( sings ( to (...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,The church has cracks in the ceiling.,( ( The church ) ( ( has ( cracks ( in ( the c...,(ROOT (S (NP (DT The) (NN church)) (VP (VBZ ha...
1,"[entailment, entailment, entailment, neutral, ...",2677109430.jpg#1,entailment,2677109430.jpg#1r1e,This church choir sings to the masses as they ...,( ( This ( church choir ) ) ( ( ( sings ( to (...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,The church is filled with song.,( ( The church ) ( ( is ( filled ( with song )...,(ROOT (S (NP (DT The) (NN church)) (VP (VBZ is...
2,"[contradiction, contradiction, contradiction, ...",2677109430.jpg#1,contradiction,2677109430.jpg#1r1c,This church choir sings to the masses as they ...,( ( This ( church choir ) ) ( ( ( sings ( to (...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,A choir singing at a baseball game.,( ( ( A choir ) ( singing ( at ( a ( baseball ...,(ROOT (NP (NP (DT A) (NN choir)) (VP (VBG sing...
3,"[neutral, neutral, neutral, neutral, neutral]",6160193920.jpg#4,neutral,6160193920.jpg#4r1n,"A woman with a green headscarf, blue shirt and...",( ( ( A woman ) ( with ( ( ( ( ( a ( green hea...,(ROOT (NP (NP (DT A) (NN woman)) (PP (IN with)...,The woman is young.,( ( The woman ) ( ( is young ) . ) ),(ROOT (S (NP (DT The) (NN woman)) (VP (VBZ is)...
4,"[entailment, entailment, contradiction, entail...",6160193920.jpg#4,entailment,6160193920.jpg#4r1e,"A woman with a green headscarf, blue shirt and...",( ( ( A woman ) ( with ( ( ( ( ( a ( green hea...,(ROOT (NP (NP (DT A) (NN woman)) (PP (IN with)...,The woman is very happy.,( ( The woman ) ( ( is ( very happy ) ) . ) ),(ROOT (S (NP (DT The) (NN woman)) (VP (VBZ is)...


In [ ]:

# РЕЖИМ 1: предсказание слов, закрытых токеном MASK в фразе. На вход нейросети надо подать фразу в формате: [CLS] Я пришел в [MASK] и купил [MASK]. [SEP]

# входная фраза с закрытыми словами с помощью [MASK]
sentence = 'Я пришел в [MASK] и купил [MASK].'  #@param {type:"string"}

print(sentence)


#-------------------------
# преобразование в токены (tokenizer.tokenize() не обрабатывает [CLS], [MASK], поэтому добавим их вручную)
sentence = sentence.replace(' [MASK] ','[MASK]'); sentence = sentence.replace('[MASK] ','[MASK]'); sentence = sentence.replace(' [MASK]','[MASK]')  # удаляем лишние пробелы
sentence = sentence.split('[MASK]')             # разбиваем строку по маске
tokens = ['[CLS]']                              # фраза всегда должна начинаться на [CLS]
# обычные строки преобразуем в токены с помощью tokenizer.tokenize(), вставляя между ними [MASK]
for i in range(len(sentence)):
    if i == 0:
        tokens = tokens + tokenizer.tokenize(sentence[i]) 
    else:
        tokens = tokens + ['[MASK]'] + tokenizer.tokenize(sentence[i]) 
tokens = tokens + ['[SEP]']                     # фраза всегда должна заканчиваться на [SEP] 
# в tokens теперь токены, которые гарантированно по словарю преобразуются в индексы
#-------------------------
#print(tokens)

# преобразуем в массив индексов, который можно подавать на вход сети, причем число 103 в нем это [MASK]
token_input = tokenizer.convert_tokens_to_ids(tokens)        
#print(token_input)
# удлиняем до 512 длины
token_input = token_input + [0] * (512 - len(token_input))


# создаем маску, заменив все числа 103 на 1, а остальное 0
mask_input = [0]*512
for i in range(len(mask_input)):
    if token_input[i] == 103:
        mask_input[i] = 1
#print(mask_input)

# маска фраз (вторая фраза маскируется числом 1, а все остальное числом 0)
seg_input = [0]*512


# конвертируем в numpy в форму (1,) -> (1,512)
token_input = np.asarray([token_input])
mask_input = np.asarray([mask_input])
seg_input = np.asarray([seg_input])


# пропускаем через нейросеть...
predicts = model.predict([token_input, seg_input, mask_input])[0]       # в [0] полная фраза с заполненными предсказанными словами на месте [MASK]
predicts = np.argmax(predicts, axis=-1)


# форматируем результат в строку, разделенную пробелами
predicts = predicts[0][:len(tokens)]    # длиной как исходная фраза (чтобы отсечь случайные выбросы среди нулей дальше)
out = []
# добавляем в out только слова в позиции [MASK], которые маскированы цифрой 1 в mask_input
for i in range(len(mask_input[0])):
    if mask_input[0][i] == 1:           # [0][i], т.к. требование было (1,512)
        out.append(predicts[i]) 

out = tokenizer.convert_ids_to_tokens(out)      # индексы в токены
out = ' '.join(out)                             # объединяем в одну строку с пробелами
out = tokenization.printable_text(out)          # в читабельную версию
out = out.replace(' ##','')                     # объединяем раздъединенные слова "при ##шел" -> "пришел"
print('Result:', out)

In [3]:

# РЕЖИМ 2: проверка логичности двух фраз. На вход нейросети надо подать фразу в формате: [CLS] Я пришел в магазин. [SEP] И купил молоко. [SEP]

sentence_1 = 'Children smiling and waving at camera'      #@param {type:"string"}
sentence_2 = 'They are smiling at their parents'          #@param {type:"string"}


print(sentence_1, '->', sentence_2)

# строки в массивы токенов
tokens_sen_1 = tokenizer.tokenize(sentence_1)
tokens_sen_2 = tokenizer.tokenize(sentence_2)

tokens = ['[CLS]'] + tokens_sen_1 + ['[SEP]'] + tokens_sen_2 + ['[SEP]']
#print(tokens)

# преобразуем строковые токены в числовые индексы:
token_input = tokenizer.convert_tokens_to_ids(tokens)  
# удлиняем до 512      
token_input = token_input + [0] * (512 - len(token_input))

# маска в этом режиме все 0
mask_input = [0] * 512

# в маске предложений под второй фразой, включая конечный SEP, надо поставить 1, а все остальное заполнить 0
seg_input = [0]*512
len_1 = len(tokens_sen_1) + 2                   # длина первой фразы, +2 - включая начальный CLS и разделитель SEP
for i in range(len(tokens_sen_2)+1):            # +1, т.к. включая последний SEP
        seg_input[len_1 + i] = 1                # маскируем вторую фразу, включая последний SEP, единицами
#print(seg_input)


# конвертируем в numpy в форму (1,) -> (1,512)
token_input = np.asarray([token_input])
mask_input = np.asarray([mask_input])
seg_input = np.asarray([seg_input])


# пропускаем через нейросеть...
predicts = model.predict([token_input, seg_input, mask_input])[1]       # в [1] ответ на вопрос, является ли второе предложение логичным по смыслу
#print('Sentence is okey: ', not bool(np.argmax(predicts, axis=-1)[0]), predicts)
print('Sentence is okey:', int(round(predicts[0][0]*100)), '%')                    # [[0.9657724  0.03422766]] - левое число вероятность что второе предложение подходит по смыслу, а правое - что второе предложение случайное
out = int(round(predicts[0][0]*100)) 


Children smiling and waving at camera -> They are smiling at their parents
Sentence is okey: 99 %
